In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from wNMF import wNMF
from scipy.sparse import coo_matrix
from collections import namedtuple


# Load data
Note: class RecSys() is a stripped down version of recommender system from Week 3's programming assignment.  It loads the rating matrix from raw data and calculates RMSE

In [2]:
MV_users = pd.read_csv('movie_data/users.csv')
MV_movies = pd.read_csv('movie_data/movies.csv')
train = pd.read_csv('movie_data/train.csv')
test = pd.read_csv('movie_data/test.csv')

Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [3]:
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        self.Mm=None
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies)))

    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID]
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(train.rating)
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())

    def rmse(self,yp):
        # yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())

In [4]:
rs = RecSys(data)
rs.Mr.shape

(6040, 3883)

---
# Section 1

We can use Non-negative Matrix Factorization to impute missing values in the ratings matrix. The simplest method is to run NMF to factor the ratings matrix $X$ into $W H$, then multiply $W H$ to recover the transformed ratings matrix $X'$.  The entries of $X'$ represent predicted ratings.

While this method is conceptually simple, it can be effective.  But as you will see, its effectiveness is highly dependent on the details of its implementation.

In [5]:
nmf = NMF(n_components=len(rs.genres),
          init="nndsvda",
          max_iter=1000,
          random_state=1).fit(rs.Mr)

In [6]:
W = nmf.transform(rs.Mr)
H = nmf.components_

In [7]:
X_from_nmf = nmf.inverse_transform(W)

In [8]:
yp_nmf=[]
for i in range(len(rs.data.test)):
    x = rs.data.test.iloc[i]
    mid = x.mID
    uid = x.uID
    yp_nmf.append(X_from_nmf[rs.uid2idx[uid],rs.mid2idx[mid]])

print(f"RSME= {rs.rmse(yp_nmf)}")

RSME= 2.8568162524287777





| Method                              |             RMSE              |
|:------------------------------------|:-----------------------------:|
| $\color{red}{\textbf{NMF}}$         | $\color{red}{\textbf{2.857}}$ |
| Baseline, $Y_p$=3                   |             1.259             |
| Baseline, $Y_p=\mu_u$               |             1.035             |
| Collaborative, cosine               |             1.026             |
| Content based, item-item            |             1.012             |
| Collaborative, jaccard, $M_r\geq 1$ |             0.991             |
| Collaborative, jaccard, $M_r\geq 3$ |             0.982             |
| Collaborative, jaccard, $M_r$       |             0.951             |

---
# Section 2

The RMSE for our predictions was 2.857, which is absolutely terrible in the context of this problem.  A recommender system which implemented this model would be worthless.

The RMSE for this recommender system is the mean error of our predictions.  RMSE is similar to the mean absolute error (MAE), except that larger errors have a greater influence on RMSE (because they are squared).  RMSE cannot be interpreted as literally as MAE, so it's worth comparing this RMSE value to the RMSE for other recommender system models.

The movie rating is on a scale of 1-5, so we could get a better RMSE from a model that blindly assigned a 3 for every unknown rating. This hypothetical recommender system would have an RMSE no worse than 2, since the largest possible squared error for a single prediction would be 4 (if the true rating was a 5 or 1).  Hopefully this context makes it clear that the Non-negative matrix factorization yielded *terrible* predictions.

In [9]:
#calculate sparsity of ratings matrix
print(f"sparsity= {1-(np.count_nonzero(rs.Mr)/rs.Mr.size)}")

sparsity= 0.9701472542053747


#### Why did NMF fail so spectacularly?

It isn't surprising that the NMF performed so poorly.  97% of the matrix were 0's, which are intended to be placeholders that indicate missing ratings.  However, sklearn's NMF implementation doesn't know to ignore the zeroes.  This NMF implementation is doomed to fail as a recommender system, because it is fundamentally mis-interpreting the missing ratings.

We could try using this NMF as a starting point for a collaborative filtering model, but anything built on top of this NMF implementation will suffer.

Instead, we can fix the NMF-based recommender system by using an implementation that ignores missing values.  Specifically, we can use a weighted-NMF (wNMF) implementation.

WNMF is similar to NMF, except that wNMF assigns weights to matrix entries.  If we assign zero weight to missing ratings, the wNMF will factor the rating matrix in a way that only considers minimizing the error of the known rating values.

In [10]:
#### wNMF implementation

In [11]:
rs_Mr = rs.Mr.astype(float)
weights = (rs.Mr > 0).astype(float)

Note: the wNMF takes a long time to run (~1 hour), so I saved the result as `wnmf_15runs.pkl` and disabled the following cell, but you're welcome to uncomment and run it yourself if you'd like

In [12]:
# wnmf = wNMF(n_components=len(rs.genres),
#            max_iter=1000).fit(rs_Mr.T, weights.T, n_run=15)

In [13]:
import pickle
## save wnmf as .pkl
# with open('wnmf_15runs.pkl','wb') as f:
#     pickle.dump(wnmf,f)

# load wnmf from .pkl
with open('wnmf_15runs.pkl', 'rb') as f:
    wnmf = pickle.load(f)

In [14]:
H_ = wnmf.U.T
print(H_.shape)

(18, 3883)


In [15]:
W_ = wnmf.V.T
print(W_.shape)

(6040, 18)


In [16]:
# reconstruct the ratings matrix
X_from_wnmf = W_ @ H_

In [17]:
# calculate RMSE of the wNMF-based recommender system
yp_wnmf = []
for i in range(len(rs.data.test)):
    x = rs.data.test.iloc[i]
    mid=x.mID
    uid=x.uID
    yp_wnmf.append(
        X_from_wnmf[rs.uid2idx[uid],rs.mid2idx[mid]]
    )
print(f"RSME= {rs.rmse(yp_wnmf)}")

RSME= 0.9565517407741765


#### wNMF conclusion

The wNMF has an RMSE that is significantly better than sklearn's NMF implementation.  In fact, the wNMF model was close to beating the best collaborative filtering model that we implemented in the Week 3 lab.

Not bad!

| Method                              | RMSE  |
|:------------------------------------|:-----:|
| Baseline, $Y_p$=3                   | 1.259 |
| Baseline, $Y_p=\mu_u$               | 1.035 |
| Collaborative, cosine               | 1.026 |
| Content based, item-item            | 1.012 |
| Collaborative, jaccard, $M_r\geq 1$ | 0.991 |
| Collaborative, jaccard, $M_r\geq 3$ | 0.982 |
| $\color{red}{\textbf{wNMF}}$        | $\color{red}{\textbf{0.957}}$  |
| Collaborative, jaccard, $M_r$       | 0.951 |


